In [1]:
import heapq

class UniformCostSearch:
    def __init__(self, initial_state, n) -> None:
        self.n = n
        self.initial_state = initial_state
        self.explored = set()

    def does_conflict(self, current_state):
        for i in range(len(current_state)):
            for j in range(i + 1, len(current_state)):
                if current_state[i] == current_state[j] or abs(current_state[i] - current_state[j]) == j - i:
                    return True
        return False

    def generate_successors(self, parent_state):
        new_state_set = []
        for i in range(self.n):
            for j in range(self.n):
                if j != parent_state[i]:
                    new_state = parent_state.copy()
                    new_state[i] = j
                    new_state_set.append(new_state)
        return new_state_set


    def solve(self):
        frontier = [(0, self.initial_state)]
        explored = set()
        while frontier:
            cost, current_state = heapq.heappop(frontier)
            if not self.does_conflict(current_state):
                return current_state
            explored.add(tuple(current_state))
            for successor in self.generate_successors(current_state):
                if tuple(successor) not in explored:
                    heapq.heappush(frontier, (cost+1, successor))
        return None

In [2]:
import heapq

class AStarSearch:
    def __init__(self, initial_state, n) -> None:
        self.n = n
        self.initial_state = initial_state
        self.explored = set()

    def count_conflicts(self, current_state):
        count = 0
        for i in range(len(current_state)):
            for j in range(i + 1, len(current_state)):
                if current_state[i] == current_state[j] or abs(current_state[i] - current_state[j]) == j - i:
                    count += 1
        return count
    
    def does_conflict(self, current_state):
        for i in range(len(current_state)):
            for j in range(i + 1, len(current_state)):
                if current_state[i] == current_state[j] or abs(current_state[i] - current_state[j]) == j - i:
                    return True
        return False
    
    def generate_successors(self, parent_state):
        new_state_set = []
        for i in range(self.n):
            for j in range(self.n):
                if j != parent_state[i]:
                    new_state = parent_state.copy()
                    new_state[i] = j
                    new_state_set.append(new_state)
        return new_state_set
    
    def solve(self):
        frontier = [(self.count_conflicts(self.initial_state), 0, self.initial_state)]
        explored=set()
        while frontier:
            heuristic, cost, current_state = heapq.heappop(frontier)
            if not self.does_conflict(current_state):
                return current_state
            self.explored.add(tuple(current_state))
            for successor in self.generate_successors(current_state):
                if tuple(successor) not in self.explored:
                    heapq.heappush(frontier, (self.count_conflicts(successor),cost+1+self.count_conflicts(successor), successor))
        return None

In [3]:
import random
import heapq

class GeneticAlgorithmSearch:
    def __init__(self, initial_state, board_size):
        self.board_size = board_size
        self.initial_state = initial_state
        self.generation = 0

    def count_conflicts(self, current_state):
        count = 0
        for i in range(len(current_state)):
            for j in range(i + 1, len(current_state)):
                if current_state[i] == current_state[j] or abs(current_state[i] - current_state[j]) == j - i:
                    count += 1
        return count
    
    def initialize_population(self, population_size):
        population = []
        num = random.randint(2, population_size)
        while num != 0:
            temp = [random.randint(0, self.board_size - 1) for _ in range(self.board_size)]
            heapq.heappush(population, (self.count_conflicts(temp),temp))
            num -= 1
        return population

    def goal_test(self, population):
        for individual in population:
            if self.count_conflicts(individual[1]) == 0:
                return individual[1]
        return None

    def randomize(self, population):
        new_population = []
        n = random.randint(2, len(population))
        for i in range(n):
            new_population.append(population[i][1])
        return new_population

    def crossover(self, parent1, parent2):
        parent_length = len(parent1)
        crossover_point = random.randint(0, parent_length - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:parent_length]
        child2 = parent2[:crossover_point] + parent1[crossover_point:parent_length]
        return child1, child2

    def mutate(self, state):
        state_length = len(state)
        mutate_point = random.randint(0, state_length - 1)
        mutation = random.randint(0, state_length - 1)
        state[mutate_point] = mutation
        return state
    
    def solve(self):
        population = self.initialize_population(self.board_size)
        heapq.heappush(population, (self.count_conflicts(self.initial_state),self.initial_state))
        result = self.goal_test(population)

        while result is None:
            random_population = self.randomize(population)
            for i in range(0, len(random_population), 2):
                if i + 2 <= len(random_population):
                    random_population[i], random_population[i + 1] = self.crossover(random_population[i], random_population[i + 1])
                    random_population[i] = self.mutate(random_population[i])
                    random_population[i + 1] = self.mutate(random_population[i + 1])

            for board in random_population:
                population.pop()
            for board in random_population:
                heapq.heappush(population, (self.count_conflicts(board),board))
            result = self.goal_test(population)

        return result

In [4]:
def print_chessboard(state, n):
    chessboard = [['.' for j in range(n)] for i in range(n)]
    for i in range(n):
        chessboard[state[i]][i] = 'Q'
    for i in range(n):
        for j in range(n):
            print(chessboard[i][j], end=' ')
        print()

In [5]:
import time,random

def main():    
    print("1. Uniform Cost Search")
    print("2. A* Search")
    print("3. Genetic Algorithm Search")
    choice = int(input("Your choice: "))
    n = int(input("Enter number of queens: "))
    print('')
    initial_state = [random.randint(0, n - 1) for _ in range(n)]
    print('Initial state:')
    print_chessboard(initial_state, n) 
    print()
    start_time = time.time()
    if choice == 1:
        solution = UniformCostSearch(initial_state,n).solve()
    if choice == 2:
        solution = AStarSearch(initial_state,n).solve()
    if choice == 3:
        solution = GeneticAlgorithmSearch(initial_state,n).solve()
    run_time=(time.time() - start_time)*1000
             
    if solution is None:
        print('No solutions')
    else:
        print('Solution:')
        print_chessboard(solution, n)         
    print()
    print(f'running time: {run_time:.4f} miliseconds')
    


In [ ]:
%pip install memory_profiler
%load_ext memory_profiler

%memit main()

Note: you may need to restart the kernel to use updated packages.
1. Uniform Cost Search
2. A* Search
3. Genetic Algorithm Search
